In [ ]:
import pandas as pd

import yaml
import json
import os
import sys
import warnings
from dotenv import load_dotenv

load_dotenv()
REPO_PATH = os.getenv("REPO_PATH")
sys.path.insert(0, rf'{REPO_PATH}src')
warnings.filterwarnings("ignore")

from utils.forecast_utils import RNNGenerator
from utils.model_utils import optimize_hyperparameters

### Import data

In [ ]:
# load yaml file
with open('variable_config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

# Define feature list and target
feature_columns = [
    'REALIZED_VOL',
    *config['BASE'],
    *config['TEMPORAL'],
    *config['SENTIMENT_CUSTOM']
]


In [ ]:
DATA_PARAMS: dict[str, any] = {
    'feature_columns': feature_columns,
    'target_column': 'REALIZED_VOL',
    'window_size': 14,
    'test_size': 0.2,
    'val_size': 0.2,
    'CV': False,
    'scaler_type': 'RobustScaler'
}

TRIAL_PARAMS = {
        'units_first_layer': [16, 32, 64, 128],
        'units_second_layer': [16, 32, 64, 96],
        'l2_strength': [1e-5, 1e-3],
        'learning_rate': [1e-5, 1e-2],
        'batch_size': [16, 32, 64],
        'noise_std': [0.01, 0.1]
}

# Specify data
FUTURE = 'CLc1'
RNN_TYPE='BiLSTM'

# Find the best hyperparameters using Optuna
best_params = optimize_hyperparameters(
    FUTURE,
    TRIAL_PARAMS,
    DATA_PARAMS,
    RNN_TYPE,
    n_trials=50, 
    n_jobs=-1
)

current_dt = pd.Timestamp.now().strftime('%Y.%m.%d_%H.%M')
hp_filename = f'{FUTURE}_{RNN_TYPE}_{current_dt}'

print(f'Best parameters for {RNN_TYPE} model: ', best_params)

# Save best hyperparameters to .json file
with open(f'hyperpm_archive/{hp_filename}.json', 'w') as file:
    json.dump(best_params, file, indent=4)
